In [ ]:
import img2dataset

url_list = r"C:\Users\swtir\OneDrive\Documents\Deep_Machine_Learning_ease_of_working_locally\deep-machine-learning\Final_Project\laion1B-nolang-aesthetic"
# Download and process images
img2dataset.download(
    url_list=url_list,
    input_format="parquet",
    url_col="URL",
    caption_col="TEXT",
    output_format="files",
    output_folder="./images",
    processes_count=16,
    thread_count=64,
    image_size=512,
    resize_only_if_bigger=True,
    save_additional_columns=["aesthetic", "pwatermark", "punsafe"]
)

Starting the downloading of this file
Sharding file number 1 of 128 called C:/Users/swtir/OneDrive/Documents/Deep_Machine_Learning_ease_of_working_locally/deep-machine-learning/Final_Project/laion1B-nolang-aesthetic/part-00000-a718cdfa-8fa6-4f99-a950-2ffa6b13c6c4-c000.snappy.parquet


0it [00:00, ?it/s]

File sharded in 22 shards
Sharding file number 2 of 128 called C:/Users/swtir/OneDrive/Documents/Deep_Machine_Learning_ease_of_working_locally/deep-machine-learning/Final_Project/laion1B-nolang-aesthetic/part-00001-a718cdfa-8fa6-4f99-a950-2ffa6b13c6c4-c000.snappy.parquet
File sharded in 22 shards


20it [12:01:36, 2485.35s/it]

Sharding file number 3 of 128 called C:/Users/swtir/OneDrive/Documents/Deep_Machine_Learning_ease_of_working_locally/deep-machine-learning/Final_Project/laion1B-nolang-aesthetic/part-00002-a718cdfa-8fa6-4f99-a950-2ffa6b13c6c4-c000.snappy.parquet
File sharded in 22 shards


42it [18:18:39, 499.07s/it] 

Sharding file number 4 of 128 called C:/Users/swtir/OneDrive/Documents/Deep_Machine_Learning_ease_of_working_locally/deep-machine-learning/Final_Project/laion1B-nolang-aesthetic/part-00003-a718cdfa-8fa6-4f99-a950-2ffa6b13c6c4-c000.snappy.parquet
File sharded in 22 shards


48it [18:28:34, 153.51s/it]

In [3]:
import os
import shutil

# Define the root directory where the subfolders are located
root_dir = r"C:\Users\swtir\OneDrive\Documents\Deep_Machine_Learning_ease_of_working_locally\deep-machine-learning\Final_Project\images"

# Define the destination directory where all files will be merged
destination_dir = r"C:\Users\swtir\OneDrive\Documents\Deep_Machine_Learning_ease_of_working_locally\deep-machine-learning\Final_Project\images2"

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Walk through all subfolders and move files to the destination directory
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        # Construct the full file path
        file_path = os.path.join(subdir, file)
        
        # Move each file to the destination folder
        shutil.move(file_path, os.path.join(destination_dir, file))

print(f"All files have been moved to {destination_dir}")

All files have been moved to C:\Users\swtir\OneDrive\Documents\Deep_Machine_Learning_ease_of_working_locally\deep-machine-learning\Final_Project\images2


In [3]:
from PIL import Image
from datasets import Dataset
import pandas as pd

# Define the directory where the images and captions are stored
data_dir = r"C:\Users\swtir\OneDrive\Documents\Deep_Machine_Learning_ease_of_working_locally\deep-machine-learning\Final_Project\images2"

# Step 1: Collect data from the directory
def collect_data(data_dir):
    data_entries = []

    # Iterate over the files in the directory
    for filename in os.listdir(data_dir):
        # We only care about the base filename (without extension) to match image and txt
        if filename.endswith(".jpg"):
            base_filename = os.path.splitext(filename)[0]
            
            # Construct the paths for the image and caption
            image_path = os.path.join(data_dir, f"{base_filename}.jpg")
            caption_path = os.path.join(data_dir, f"{base_filename}.txt")
            
            # Read the image
            image = Image.open(image_path).convert("RGB")
            
            # Read the caption with encoding handling
            caption = ""
            try:
                # Try reading as utf-8 first
                with open(caption_path, "r", encoding="utf-8") as f:
                    caption = f.read().strip()
            except UnicodeDecodeError:
                # Fallback to 'latin1' or 'ISO-8859-1' encoding
                with open(caption_path, "r", encoding="latin1") as f:
                    caption = f.read().strip()
            
            # Create an entry combining the image path and the caption
            data_entries.append({
                "image_path": image_path,
                "caption": caption
            })
    
    return data_entries

# Step 2: Collect the data
data = collect_data(data_dir)

# Step 3: Convert the data to a pandas DataFrame (or use directly with Hugging Face Dataset)
df = pd.DataFrame(data)

# Step 4: Convert the DataFrame to Hugging Face Dataset (if needed)
dataset = Dataset.from_pandas(df)

# Step 5: Verify the dataset structure by printing a sample
print(dataset[0])

{'image_path': 'C:\\Users\\swtir\\OneDrive\\Documents\\Deep_Machine_Learning_ease_of_working_locally\\deep-machine-learning\\Final_Project\\images2\\000050205.jpg', 'caption': 'Best Ever No-Bake Cheesecake'}


In [5]:
# Step 1: Save the DataFrame (df)
df.to_pickle("collected_data.pkl")  # Save DataFrame as a pickle file
df.to_csv("collected_data.csv", index=False)  # Alternatively, save as a CSV file

# Step 2: Save the Hugging Face Dataset
dataset.save_to_disk("huggingface_dataset")  # Save the dataset to disk

Saving the dataset (0/1 shards):   0%|          | 0/346339 [00:00<?, ? examples/s]